In [1]:
import warnings

import pandas as pd 
import numpy as np

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from mypipes import *

In [2]:
warnings.filterwarnings('ignore')

In [3]:
train_file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/loan_data_train.csv'
test_file=r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)               

In [4]:
ld_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              2199 non-null   float64
 1   Amount.Requested                2199 non-null   object 
 2   Amount.Funded.By.Investors      2199 non-null   object 
 3   Interest.Rate                   2200 non-null   object 
 4   Loan.Length                     2199 non-null   object 
 5   Loan.Purpose                    2199 non-null   object 
 6   Debt.To.Income.Ratio            2199 non-null   object 
 7   State                           2199 non-null   object 
 8   Home.Ownership                  2199 non-null   object 
 9   Monthly.Income                  2197 non-null   float64
 10  FICO.Range                      2200 non-null   object 
 11  Open.CREDIT.Lines               2196 non-null   object 
 12  Revolving.CREDIT.Balance        21

In [5]:
ld_train.sample(10)

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
469,14958.0,5000,5000,11.86%,36 months,major_purchase,4%,TX,RENT,5577.00,715-719,3,1069,0.0,1 year
1411,82746.0,8000,8000,15.80%,36 months,credit_card,10.76%,OK,MORTGAGE,4583.33,675-679,13,4084,3.0,2 years
1842,33055.0,2200,2200,11.71%,36 months,home_improvement,1.02%,NY,MORTGAGE,7167.92,725-729,4,2639,0.0,3 years
820,15631.0,4500,4500,8.88%,36 months,credit_card,7.07%,OR,MORTGAGE,5250.00,710-714,10,9356,2.0,9 years
1118,71782.0,14500,14500,13.11%,36 months,debt_consolidation,28.29%,WA,MORTGAGE,5988.67,700-704,12,18020,2.0,10+ years
133,36576.0,3000,3000,7.90%,36 months,credit_card,22.03%,PA,MORTGAGE,5583.33,705-709,15,12968,0.0,3 years
416,86783.0,25000,25000,18.49%,60 months,debt_consolidation,10.08%,CA,RENT,7666.67,715-719,12,14631,2.0,3 years
497,61151.0,8000,8000,17.77%,36 months,credit_card,34.56%,IL,RENT,3291.67,725-729,26,18518,0.0,2 years
1322,78960.0,11000,11000,12.12%,36 months,debt_consolidation,5.83%,CA,MORTGAGE,4583.33,695-699,11,12361,0.0,10+ years
1879,27371.0,7400,7400,13.99%,36 months,other,20.52%,NY,RENT,4166.25,675-679,17,6653,1.0,NaN


In [6]:
ld_train['Employment.Length'].value_counts()

# dont need : ID , Intereste.Rate

# Amount requested : V1
# Amount.Funded.By.Investors : V2 
# Open.CREDIT.Lines : V3
# Revolving.CREDIT.Balance : V4

# convert it to numeric , and then impute missing values with median

# Debt.To.Income.Ratio : V5

# remove percentage sign , covert to numeric and then impute with median

# Loan.Length ,Loan.Purpose,State,Home.Ownership,Employment.Length
# V6-V10
# create dummies for these, considering frequency cutoff as 20

# Monthly.Income : V11,
# Inquiries.in.the.Last.6.Months :V12

# impute missing values with median

# FICO.Range : V13

# split the column at '-', convert the resulting columns to numeric(say a ,b) , 
# then create new column = (a+b)/2 


10+ years    575
< 1 year     229
2 years      217
3 years      203
5 years      181
4 years      162
1 year       159
6 years      134
7 years      109
8 years       95
9 years       66
.              1
Name: Employment.Length, dtype: int64

In [7]:
# pipe_name=pdPipeline([
#     ('name of the process',call to process),
#     ('name of the process', call to process),
#     .....
# ])

In [8]:
p1=pdPipeline([
    ('var_select',VarSelector(['Amount.Requested','Amount.Funded.By.Investors',
                               'Open.CREDIT.Lines','Revolving.CREDIT.Balance'])),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])



In [9]:
p2=pdPipeline([
    ('var_select',VarSelector(['Debt.To.Income.Ratio'])),
    ('string_clean',string_clean(replace_it='%',replace_with='')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

In [10]:
p3=pdPipeline([
    ('var_select',VarSelector(['Loan.Length', 'Loan.Purpose','State','Home.Ownership',
                               'Employment.Length'])),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(20))
])

In [11]:
p4=pdPipeline([
    ('var_select',VarSelector(['Monthly.Income','Inquiries.in.the.Last.6.Months'])),
    ('missing_trt',DataFrameImputer())
])

In [12]:
p5=pdPipeline([
    ('var_select',VarSelector(['FICO.Range'])),
    ('custom_fico',custom_fico()),
    ('missing_trt',DataFrameImputer())
])

In [13]:
p6 = pdPipeline([
    ('var_select', VarSelector(['Employment.Length'])),
    ('custom_El', custom_El()),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt', DataFrameImputer())
])

In [14]:
data_pipe=FeatureUnion([
    ('obj_to_num',p1),
    ('dtir',p2),
    ('obj_to_dum',p3),
    ('num',p4),
    ('fico',p5),
    ('el', p6)
])

In [15]:
data_pipe.fit(ld_train)

FeatureUnion(transformer_list=[('obj_to_num',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Amount.Requested',
                                                                              'Amount.Funded.By.Investors',
                                                                              'Open.CREDIT.Lines',
                                                                              'Revolving.CREDIT.Balance'])),
                                                  ('convert_to_numeric',
                                                   convert_to_numeric()),
                                                  ('missing_trt',
                                                   DataFrameImputer())])),
                               ('dtir',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=...
 

In [16]:
len(data_pipe.get_feature_names())

62

In [17]:
data_pipe.transform(ld_train).shape
data_pipe

FeatureUnion(transformer_list=[('obj_to_num',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['Amount.Requested',
                                                                              'Amount.Funded.By.Investors',
                                                                              'Open.CREDIT.Lines',
                                                                              'Revolving.CREDIT.Balance'])),
                                                  ('convert_to_numeric',
                                                   convert_to_numeric()),
                                                  ('missing_trt',
                                                   DataFrameImputer())])),
                               ('dtir',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=...
 

In [18]:
x_train=pd.DataFrame(data=data_pipe.transform(ld_train),
                    columns=data_pipe.get_feature_names())

In [19]:
x_test=pd.DataFrame(data=data_pipe.transform(ld_test),
                    columns=data_pipe.get_feature_names())

In [20]:
x_train.shape

(2200, 62)

In [21]:
x_test.shape

(300, 62)

In [22]:
x_train.columns

Index(['obj_to_num__Amount.Requested',
       'obj_to_num__Amount.Funded.By.Investors',
       'obj_to_num__Open.CREDIT.Lines', 'obj_to_num__Revolving.CREDIT.Balance',
       'dtir__Debt.To.Income.Ratio', 'obj_to_dum__Loan.Length_36 months',
       'obj_to_dum__Loan.Length_60 months',
       'obj_to_dum__Loan.Purpose_debt_consolidation',
       'obj_to_dum__Loan.Purpose_credit_card',
       'obj_to_dum__Loan.Purpose_other',
       'obj_to_dum__Loan.Purpose_home_improvement',
       'obj_to_dum__Loan.Purpose_major_purchase',
       'obj_to_dum__Loan.Purpose_small_business',
       'obj_to_dum__Loan.Purpose_car', 'obj_to_dum__Loan.Purpose_wedding',
       'obj_to_dum__Loan.Purpose_medical', 'obj_to_dum__Loan.Purpose_moving',
       'obj_to_dum__State_CA', 'obj_to_dum__State_NY', 'obj_to_dum__State_FL',
       'obj_to_dum__State_TX', 'obj_to_dum__State_PA', 'obj_to_dum__State_IL',
       'obj_to_dum__State_GA', 'obj_to_dum__State_NJ', 'obj_to_dum__State_VA',
       'obj_to_dum__State_MA',

In [23]:
x_test.columns

Index(['obj_to_num__Amount.Requested',
       'obj_to_num__Amount.Funded.By.Investors',
       'obj_to_num__Open.CREDIT.Lines', 'obj_to_num__Revolving.CREDIT.Balance',
       'dtir__Debt.To.Income.Ratio', 'obj_to_dum__Loan.Length_36 months',
       'obj_to_dum__Loan.Length_60 months',
       'obj_to_dum__Loan.Purpose_debt_consolidation',
       'obj_to_dum__Loan.Purpose_credit_card',
       'obj_to_dum__Loan.Purpose_other',
       'obj_to_dum__Loan.Purpose_home_improvement',
       'obj_to_dum__Loan.Purpose_major_purchase',
       'obj_to_dum__Loan.Purpose_small_business',
       'obj_to_dum__Loan.Purpose_car', 'obj_to_dum__Loan.Purpose_wedding',
       'obj_to_dum__Loan.Purpose_medical', 'obj_to_dum__Loan.Purpose_moving',
       'obj_to_dum__State_CA', 'obj_to_dum__State_NY', 'obj_to_dum__State_FL',
       'obj_to_dum__State_TX', 'obj_to_dum__State_PA', 'obj_to_dum__State_IL',
       'obj_to_dum__State_GA', 'obj_to_dum__State_NJ', 'obj_to_dum__State_VA',
       'obj_to_dum__State_MA',

In [24]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 62 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   obj_to_num__Amount.Requested                 2200 non-null   float64
 1   obj_to_num__Amount.Funded.By.Investors       2200 non-null   float64
 2   obj_to_num__Open.CREDIT.Lines                2200 non-null   float64
 3   obj_to_num__Revolving.CREDIT.Balance         2200 non-null   float64
 4   dtir__Debt.To.Income.Ratio                   2200 non-null   float64
 5   obj_to_dum__Loan.Length_36 months            2200 non-null   float64
 6   obj_to_dum__Loan.Length_60 months            2200 non-null   float64
 7   obj_to_dum__Loan.Purpose_debt_consolidation  2200 non-null   float64
 8   obj_to_dum__Loan.Purpose_credit_card         2200 non-null   float64
 9   obj_to_dum__Loan.Purpose_other               2200 non-null   float64
 10  

In [25]:
import joblib

In [26]:
joblib.dump(data_pipe, 'data_pipe.pkl')

['data_pipe.pkl']

In [31]:
file = r'/home/siddarth.jha@npci.org.in/Documents/Training/data_pipe.pkl'
data_pipe1 = joblib.load(file)
x_test1 = pd.DataFrame(data = data_pipe1.transform(ld_test),
                      columns = data_pipe1.get_feature_names())
x_test1

,obj_to_num__Amount.Requested,obj_to_num__Amount.Funded.By.Investors,obj_to_num__Open.CREDIT.Lines,obj_to_num__Revolving.CREDIT.Balance,dtir__Debt.To.Income.Ratio,obj_to_dum__Loan.Length_36 months,obj_to_dum__Loan.Length_60 months,obj_to_dum__Loan.Purpose_debt_consolidation,obj_to_dum__Loan.Purpose_credit_card,obj_to_dum__Loan.Purpose_other,...,obj_to_dum__Employment.Length_1 year,obj_to_dum__Employment.Length_6 years,obj_to_dum__Employment.Length_7 years,obj_to_dum__Employment.Length_8 years,obj_to_dum__Employment.Length_missing,obj_to_dum__Employment.Length_9 years,num__Monthly.Income,num__Inquiries.in.the.Last.6.Months,fico__fico,el__El
0,5000.0,5000.0,13.0,7686.0,12.59,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4416.67,0.0,692.0,0.0
1,18000.0,18000.0,6.0,11596.0,4.93,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5258.50,0.0,712.0,10.0
2,7200.0,7200.0,13.0,7283.0,25.16,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3750.00,0.0,752.0,6.0
3,7200.0,7200.0,14.0,4838.0,17.27,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3416.67,0.0,792.0,10.0
4,22000.0,22000.0,9.0,20181.0,18.28,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,6083.33,0.0,722.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,12000.0,11975.0,7.0,3923.0,13.91,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,7083.33,2.0,707.0,7.0
296,5000.0,5000.0,10.0,6023.0,17.22,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4000.00,0.0,707.0,10.0
297,4375.0,4375.0,9.0,6944.0,9.09,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2916.67,2.0,672.0,6.0
298,13600.0,13600.0,13.0,24383.0,12.85,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5651.33,0.0,677.0,10.0
